In [1]:
import itertools
import os
import random
import subprocess

import IPython.display
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# BASENAME_CSV = 'sorted-20241128_210446' 
BASENAME_CSV = '20241203_133243/sorted' 
I_MAP = 1

DIRECTORY_IMAGES = f'images/{BASENAME_CSV.replace("/", "_")}'
os.makedirs(DIRECTORY_IMAGES, exist_ok=True)    

df_orig = pd.read_csv(f'../logs/rundirs/{BASENAME_CSV}.csv')
df_orig

,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,...,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization
0,20241203_133245,map-generator/generated-maps/2024-11-28_13:17:...,0,HumanDrivenVehicle,783.84,6,4764.1,23,0,0,...,"(703.0, 40.6)",72.1,71,784,73,45.4,3.0,STOP@CP,"2: [782, 784]",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,20241203_133245,map-generator/generated-maps/2024-11-28_13:17:...,1,AutonomousVehicle,128.74,26,6788.3,29,0,0,...,"(184.2, 128.2)",144.5,141,254,NaN,224.7,NaN,DRIVING,1: [254],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,20241203_133245,map-generator/generated-maps/2024-11-28_13:17:...,2,AutonomousVehicle,472.77,9,9145.1,10,0,0,...,"(240.3, 264.0)",648.8,646,944,NaN,913.0,NaN,DRIVING,1: [944],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,20241203_133245,map-generator/generated-maps/2024-11-28_13:17:...,3,AutonomousVehicle,759.20,6,9528.2,6,0,0,...,"(685.0, 136.3)",426.6,425,1516,NaN,1485.8,NaN,DRIVING,1: [1516],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,20241203_133449,map-generator/generated-maps/2024-11-28_13:17:...,0,HumanDrivenVehicle,783.84,11,9092.0,11,22,22,...,"(507.5, 95.7)",486.8,485,782,NaN,750.4,NaN,DRIVING,"2: [784, 782]",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,20241203_134123,map-generator/generated-maps/2024-11-28_13:19:...,3,AutonomousVehicle,442.96,10,8993.9,11,0,0,...,"(439.3, 326.1)",148.9,148,884,NaN,853.6,NaN,DRIVING,1: [884],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
236,20241203_134345,map-generator/generated-maps/2024-11-28_13:19:...,0,HumanDrivenVehicle,779.77,11,9094.5,11,13,13,...,"(558.4, 92.2)",533.1,532,778,NaN,746.3,NaN,DRIVING,"2: [780, 778]",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
237,20241203_134345,map-generator/generated-maps/2024-11-28_13:19:...,1,AutonomousVehicle,132.88,20,5330.1,38,0,0,...,"(131.1, 152.6)",52.0,51,262,NaN,233.0,NaN,DRIVING,1: [262],0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
238,20241203_134345,map-generator/generated-maps/2024-11-28_13:19:...,2,AutonomousVehicle,328.57,11,7660.9,20,0,0,...,"(142.9, 209.7)",449.8,447,654,NaN,624.3,NaN,DRIVING,1: [654],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [3]:
df_id = df_orig['Scenario ID'].str.split(r'[;,] ', expand=True)
df_id.columns = ['filename', 'Coordination strategy', 'string_seed', 'string_probabilityForcingForHuman', 'heuristic']
df_id = pd.concat([
    df_id,
    df_id['filename'].str.extract(r'(?P<dir_map>[^/]+)/(?P<basename_scenario>[^/]+)[.]json$', expand=True),
    df_id['filename'].str.extract(r'/scenario(?P<i_map>\d+)-(?P<i_locations>\d+)[.]json$', expand=True).astype(int),
    df_id['string_probabilityForcingForHuman'].str.extract(r'^probabilityForcingForHuman (?P<probabilityForcingForHuman>[\d.]+)$', expand=True).astype(float),
], axis=1).rename(columns={'i_locations': 'Positions variant'})
df_id['filename_screenshot'] = "../map-generator/generated-maps/" + df_id['dir_map'] + '/screenshots/' + df_id['basename_scenario'] + '.png'
df_id['are_bridges'] = df_id['dir_map'].str.contains('with_bridges')
df_id['configuration'] = df_id[
    ['i_map', 'are_bridges', 'Positions variant']].agg(
    lambda r: f'map {r['i_map']}, {"with" if r['are_bridges'] else "without"} bridges, pos.var. {r['Positions variant']}', 
    axis=1
)
df_id = df_id.sort_values(['i_map', 'are_bridges', 'Positions variant'], ascending=[True, False, True])
df_id = df_id[df_id['i_map'] == I_MAP]
df_id

,filename,Coordination strategy,string_seed,string_probabilityForcingForHuman,heuristic,dir_map,basename_scenario,i_map,Positions variant,probabilityForcingForHuman,filename_screenshot,are_bridges,configuration
0,map-generator/generated-maps/2024-11-28_13:17:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-28_13:17:39_with_bridges,scenario1-1,1,1,0.0,../map-generator/generated-maps/2024-11-28_13:...,True,"map 1, with bridges, pos.var. 1"
1,map-generator/generated-maps/2024-11-28_13:17:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-28_13:17:39_with_bridges,scenario1-1,1,1,0.0,../map-generator/generated-maps/2024-11-28_13:...,True,"map 1, with bridges, pos.var. 1"
2,map-generator/generated-maps/2024-11-28_13:17:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-28_13:17:39_with_bridges,scenario1-1,1,1,0.0,../map-generator/generated-maps/2024-11-28_13:...,True,"map 1, with bridges, pos.var. 1"
3,map-generator/generated-maps/2024-11-28_13:17:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-28_13:17:39_with_bridges,scenario1-1,1,1,0.0,../map-generator/generated-maps/2024-11-28_13:...,True,"map 1, with bridges, pos.var. 1"
4,map-generator/generated-maps/2024-11-28_13:17:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-28_13:17:39_with_bridges,scenario1-1,1,1,1.0,../map-generator/generated-maps/2024-11-28_13:...,True,"map 1, with bridges, pos.var. 1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,map-generator/generated-maps/2024-11-28_13:19:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-28_13:19:18_without_bridges,scenario1-10,1,10,1.0,../map-generator/generated-maps/2024-11-28_13:...,False,"map 1, without bridges, pos.var. 10"
140,map-generator/generated-maps/2024-11-28_13:19:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-28_13:19:18_without_bridges,scenario1-10,1,10,1.0,../map-generator/generated-maps/2024-11-28_13:...,False,"map 1, without bridges, pos.var. 10"
141,map-generator/generated-maps/2024-11-28_13:19:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-28_13:19:18_without_bridges,scenario1-10,1,10,1.0,../map-generator/generated-maps/2024-11-28_13:...,False,"map 1, without bridges, pos.var. 10"
142,map-generator/generated-maps/2024-11-28_13:19:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-28_13:19:18_without_bridges,scenario1-10,1,10,1.0,../map-generator/generated-maps/2024-11-28_13:...,False,"map 1, without bridges, pos.var. 10"


In [4]:
configuration_to_filename_screenshot = {row['configuration']: row['filename_screenshot'] for _, row in df_id.iterrows()}
configuration_to_filename_screenshot

{'map 1, with bridges, pos.var. 1': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-1.png',
 'map 1, with bridges, pos.var. 2': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-2.png',
 'map 1, with bridges, pos.var. 3': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-3.png',
 'map 1, with bridges, pos.var. 4': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-4.png',
 'map 1, with bridges, pos.var. 5': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-5.png',
 'map 1, with bridges, pos.var. 6': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-6.png',
 'map 1, with bridges, pos.var. 7': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-7.png',
 'map 1, with bridges, pos.var. 8': '../map-generator/generated-maps/2024-11

In [5]:
index_id = df_orig.columns.get_loc('Scenario ID')
df = pd.concat([
    df_id[['i_map', 'are_bridges', 'Positions variant', 'configuration', 
           'Coordination strategy', 'probabilityForcingForHuman']], 
    df_orig.iloc[:, index_id + 1:]], 
    axis=1
)
df

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,...,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization
0,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,0,HumanDrivenVehicle,783.84,6,...,"(703.0, 40.6)",72.1,71,784,73,45.4,3.0,STOP@CP,"2: [782, 784]",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,1,AutonomousVehicle,128.74,26,...,"(184.2, 128.2)",144.5,141,254,NaN,224.7,NaN,DRIVING,1: [254],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,2,AutonomousVehicle,472.77,9,...,"(240.3, 264.0)",648.8,646,944,NaN,913.0,NaN,DRIVING,1: [944],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,3,AutonomousVehicle,759.20,6,...,"(685.0, 136.3)",426.6,425,1516,NaN,1485.8,NaN,DRIVING,1: [1516],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,1,True,1,"map 1, with bridges, pos.var. 1",change of priorities,1.0,0,HumanDrivenVehicle,783.84,11,...,"(507.5, 95.7)",486.8,485,782,NaN,750.4,NaN,DRIVING,"2: [784, 782]",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1,False,10,"map 1, without bridges, pos.var. 10",change of priorities,1.0,3,AutonomousVehicle,555.12,8,...,"(585.8, 425.1)",111.9,111,1108,512,480.5,401.8,DRIVING,1: [1108],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
140,1,False,10,"map 1, without bridges, pos.var. 10",stops,1.0,0,HumanDrivenVehicle,779.77,11,...,"(558.5, 92.2)",533.2,532,778,NaN,746.4,NaN,DRIVING,"2: [780, 778]",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
141,1,False,10,"map 1, without bridges, pos.var. 10",stops,1.0,1,AutonomousVehicle,132.88,23,...,"(100.8, 157.8)",21.2,21,262,NaN,233.1,NaN,DRIVING,1: [262],0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
142,1,False,10,"map 1, without bridges, pos.var. 10",stops,1.0,2,AutonomousVehicle,368.37,11,...,"(389.5, 288.7)",65.4,65,734,NaN,704.3,NaN,DRIVING,1: [734],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [6]:
series_blocks = df[df['Vehicle type'] != 'HumanDrivenVehicle'].groupby(['configuration', 'are_bridges'], sort=False)['Is blocked'].sum()
index_blocked = series_blocks[series_blocks != 0].index
index_nonblocked = series_blocks[series_blocks == 0].index
index_nonblocked[~index_nonblocked.get_level_values('are_bridges')].get_level_values('configuration')

Index(['map 1, without bridges, pos.var. 1',
       'map 1, without bridges, pos.var. 2',
       'map 1, without bridges, pos.var. 3',
       'map 1, without bridges, pos.var. 4',
       'map 1, without bridges, pos.var. 5',
       'map 1, without bridges, pos.var. 6',
       'map 1, without bridges, pos.var. 7',
       'map 1, without bridges, pos.var. 8',
       'map 1, without bridges, pos.var. 9',
       'map 1, without bridges, pos.var. 10'],
      dtype='object', name='configuration')

In [7]:
def save_and_show(fig, basename):  # to avoid inlining large image data into the notebook file
    filename = f'{DIRECTORY_IMAGES}/{basename}.png'
    fig.savefig(filename)
    
    # The `random` is because of https://stackoverflow.com/a/43640705.
    IPython.display.display(IPython.display.HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" />'))
    
    plt.close(fig)
    
    return filename


def same_value(series):
    assert series.nunique() == 1, series
    return series.iloc[0]
    

def make_misbehaviors(df_nonbaseline):
    misbehaviors = []
    
    probabilityForcingForHuman = same_value(df_nonbaseline['probabilityForcingForHuman'])
    if probabilityForcingForHuman > 0.0:
        misbehaviors.append(f'violation of priorities ({"random" if probabilityForcingForHuman < 1.0 else "constant"})')
        
    isCanPassFirstActive = same_value(df_nonbaseline['isCanPassFirstActive'])
    if isCanPassFirstActive.startswith('hum=true, '):
        misbehaviors.append('can pass first')
    elif isCanPassFirstActive.startswith('hum=false, '):
        pass
    else:
        raise ValueError(isCanPassFirstActive)
    
    if 'probabilitySlowingDownForHuman' in df_nonbaseline.columns:
        probabilitySlowingDownForHuman = float(same_value(df_nonbaseline['probabilitySlowingDownForHuman']))
        if probabilitySlowingDownForHuman > 0.0:
            misbehaviors.append(f'moving slowly ({"random" if probabilitySlowingDownForHuman < 1.0 else "constant"})')
    
    return misbehaviors


def make_subplots(ncols):
    return plt.subplots(1, ncols, figsize=(20, 6), sharey=True, squeeze=False)
    

def plot_title(df, *, title2):
    positions = df.index.get_level_values('Positions variant').unique()
    fig, axes = make_subplots(len(positions))
    default_fig_width, default_fig_height = fig.get_size_inches()
    plt.close(fig)
    
    fig = plt.figure(figsize=(default_fig_width, 1))
    
    misbehaviors = make_misbehaviors(df[df['Coordination strategy'] != 'baseline'])
    title3 = 'Human (mis)behaviour actions: ' + ('none' if not misbehaviors else ', '.join(misbehaviors))    
    
    title = f'{title2}\n{title3}'
    fig.suptitle(title, fontsize=16)
    
    fig.tight_layout()
    title1 = '.title'
    filename_png = save_and_show(fig, f'{title1}: {title2}')
    
    return filename_png
    
    
class Formula:
    def __init__(self, label, expression=None):
        self.label = label
        self.expression = expression if expression is not None else label
        
    def __str__(self):
        return self.label.replace('`', '')
    
    def __repr__(self):
        return f'<Formula: {self.label!r}>'
    
    def apply(self, df):
        if isinstance(self.expression, str):
            return df.eval(self.expression)
        return df.apply(self.expression, axis=1)
    
    
def plot_aut_hum(df, *, title2, dfs_y1, dfs_y2, mode):
    #IPython.display.display(df)
    
    # Extract levels
    positions = df.index.get_level_values('Positions variant').unique()
    strategies = df.index.get_level_values('Coordination strategy').unique()
    
    strategy2label = {'baseline': 'baseline\n(no human effect)', 'stops': 'stops (local)'}        
    
    # Plot configuration
    fig, axes = make_subplots(len(positions))
    axes: list[list[Axes]]
    bar_width = 0.4
    
    formulas_y2_aut = [Formula('`No. of collisions`'), Formula('`No. of near-misses`')]
    colors_y2_aut = ['red', 'yellow']
    
    formulas_y2_hum = [Formula('`No. of violations`')] + formulas_y2_aut
    colors_y2_hum = ['black'] + colors_y2_aut
    
    formulas_y2_cmp = [Formula(f'`{col}` / `No. of violations`', 
                               lambda row, col=col: row[col] / row['No. of violations'] if row['No. of violations'] > 0 else 0)
                       for col in ('No. of collisions', 'No. of near-misses')]
    colors_y2_cmp = ['red', 'yellow']
    
    if mode in ('aut', 'hum'):
        column_y1 = 'Total distance traveled (m)'
        color_y1 = 'tab:blue'
        
        if mode == 'aut':
            formulas_y2 = formulas_y2_aut
            colors_y2 = colors_y2_aut
            id_vehicle_max = df['Vehicle ID'].max()
            title1 = f'Automated vehicles (summarised for AV1-AV{id_vehicle_max})' 
        else:
            formulas_y2 = formulas_y2_hum
            colors_y2 = colors_y2_hum
            title1 = 'Human-driven vehicle'
            
    elif mode == 'cmp':
        column_y1 = None
        color_y1 = None
        
        formulas_y2 = formulas_y2_cmp
        colors_y2 = colors_y2_cmp
        title1 = 'Collisions rate'
        
    else:
        raise ValueError(mode)
    
    # Get the global max values for consistent y-axis scaling
    if column_y1 is not None:
        y1_max = max(dfx[column_y1].max() for dfx in dfs_y1)
        y1_lim = y1_max * 1.1
    y2_maxes = [formula.apply(dfx).max()
                for dfx in dfs_y2
                for formula in (formulas_y2 if mode == 'cmp' else set(formulas_y2_aut + formulas_y2_hum))]
    y2_max = max(y2_maxes)
    y2_lim = y2_max * 1.1
    if mode == 'cmp':
        y2_lim = 2.0
    
    # Iterate through each Positions variant
    for i, position in enumerate(positions):
        ax: Axes = axes[0][i]
        df_pos = df.loc[position]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars
        handles = []
        if column_y1 is None:
            ax.set_yticks([])
        else:
            handles += [ax.bar(x_positions, df_pos[column_y1], width=bar_width, label=column_y1, color=color_y1)[0]]
            ax.set_xlabel('Coordination Strategy')
            ax.set_ylabel(column_y1, color=color_y1)
            ax.tick_params(axis='y', labelcolor=color_y1)
            ax.set_ylim(0, y1_lim)
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points
        handles += [ax_right.plot(x_positions, formula.apply(df_pos), label=str(formula),
                                  marker='o', linestyle='', color=color)[0]
                    for formula, color in zip(formulas_y2, colors_y2)]
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_lim)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels([strategy2label.get(s, s) for s in strategies], rotation=60, ha='right')
        ax.set_title(f'Position {position}')
        ax.grid(axis='y')
        
    fig.suptitle(title1, fontsize=16)
    
    labels = [str(x) for x in [column_y1, *formulas_y2] if x is not None]
    fig.legend(handles=handles, labels=labels, ncol=len(handles), loc='upper right')
    
    fig.tight_layout()
    filename_png = save_and_show(fig, f'{title1}: {title2}')
    
    #print(id(df))
    #IPython.display.display(df)
    
    return filename_png


key2df = {}
for are_bridges in True, False:
    for is_aut in True, False:
        dfx = df[df['are_bridges'] == are_bridges]
        dfx = dfx[dfx['configuration'].isin(index_nonblocked.get_level_values('configuration'))]
        dfx = dfx[dfx['Vehicle type'] == ('AutonomousVehicle' if is_aut else 'HumanDrivenVehicle')]
        dfx = dfx.groupby(['Positions variant', 'Coordination strategy']).agg({
            **{col: 'sum' for col in dfx.columns},
            **{col: same_value for col in (
                'Positions variant', 'Coordination strategy', 'configuration',
                'probabilityForcingForHuman', 'isCanPassFirstActive', 'probabilitySlowingDownForHuman',
            ) if col in dfx.columns},
            **{col: 'max' for col in ('Vehicle ID', )},
        })
        key2df[are_bridges, is_aut] = dfx    
        
for are_bridges in True, False:
    filenames_png = [
        plot_title(
            key2df[are_bridges, True],
            title2=f"Map {I_MAP} ({'with' if are_bridges else 'without'} bridges)",
        )
    ]
    for mode in 'aut', 'hum', 'cmp':
        is_aut = mode == 'aut'
        try:
            filenames_png.append(
                plot_aut_hum(
                    key2df[are_bridges, is_aut],
                    title2=f"Map {I_MAP} ({'with' if are_bridges else 'without'} bridges)", 
                    dfs_y1=[dfx for (_, is_aut_dfx), dfx in key2df.items() if is_aut_dfx == is_aut], 
                    dfs_y2=key2df.values() if mode != 'cmp' else [key2df[are_bridges, is_aut]], 
                    mode=mode,
                )
            )
        except Exception as exc:
            #print(type(exc), exc)
            raise exc
    #IPython.display.display(dfx)
    filename_out_png = f'{DIRECTORY_IMAGES}/All: Map {I_MAP} (' + ('with' if are_bridges else 'without') + ' bridges).png'
    subprocess.run(['convert', *filenames_png, '-append', filename_out_png], check=True)
    print()

In [8]:
def show_maps(title, configurations, ncols): 
    nrows = max(1, (len(configurations) + ncols - 1) // ncols)
    fig, axes_matrix = plt.subplots(nrows, ncols, figsize=(16, 9), squeeze=False)
    #print(f'{title}: {nrows}x{ncols}')

    axes = list(itertools.chain.from_iterable(axes_matrix))
    axes_matrix: list[list[Axes]]
    assert len(axes) >= len(configurations)
    
    for ax in axes:
        ax.axis('off')
        
    for ax, configuration in zip(axes, configurations):
        filename_screenshot = configuration_to_filename_screenshot[configuration]
        try:
            image = plt.imread(filename_screenshot)
            ax.imshow(image)
        except Exception as exc:
            print(exc)
        ax.title.set_text(f'Configuration:\n{configuration}')
    
    fig.suptitle(title, fontsize=16)    
    fig.tight_layout()
    fig.subplots_adjust(wspace=0.1, hspace=0.3)
    save_and_show(fig, title)
    
    
for title, index in {'Non-blocked': index_nonblocked, 'Blocked': index_blocked}.items():
    for are_bridges in True, False:
        show_maps(f'{title}: Map {I_MAP} ({"with" if are_bridges else "without"} bridges)', 
                  index[index.get_level_values('are_bridges') == are_bridges].get_level_values('configuration'), 
                  4)